# Problem set 3: Loading and structuring data from Denmark Statistics

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import ipywidgets as widgets

# Tasks

## Create a pandas DataFrame

Modify the code below such that *income* and *consumption* are variables in the *dt* DataFrame.

In [ ]:
np.random.seed(1999)
 
N = 100
mydata = {}
mydata['id'] = range(N)
income = np.exp(np.random.normal(size=N))
mydata['income'] = income
consumption = np.sqrt(income)
mydata['consumption'] = consumption

dt = pd.DataFrame(mydata)
dt.head()

**Answer:** see A01.py

## Create new variable

1) Add a new variable *ratio* which is the ratio of consumption to income.

In [ ]:
# write your code here
dt['ratio'] = dt.consumption/dt.income
dt.head()

**Answer:** See A02.py

## Summary statistics

Produce summary statistics using `.describe()`.

In [ ]:
# write your code here
dt.describe()


**Answer:** See A03.py

## Indexing

Select everybody with an income above 1.

In [ ]:
# write your code here
I = dt.income > 1
dt.loc[I,:].head()

**Answer:** See A04.py

Select everybody with an income *above* 1 and a ratio *above* 0.7.

In [ ]:
# write your code 
I = (dt.income>1) & (dt.ratio>0.7)
dt.loc[I,:]

**Answer:** See A05.py

Set consumption equal to 0.5 if income is less than 0.5.

In [ ]:
# write your code here
I = dt.consumption<0.5
dt.loc[I, ['consumption']] = 0.5
dt['consumption'].mean() # <- compare with answer

**Answer:**  See A06.py

Set consumption equal to income if income is less than 0.5.

In [ ]:
# write your code here
I = dt.consumption<=0.5
dt.loc[I,['consumption']]  = dt.loc[I,['income']].values
dt['consumption'].mean() # <- compare with answer

**Answer:** See A07.py

## Dropping

Drop the *ratio* variable and all rows with an income above 1.5. After this, also drop the first 5 rows.

In [ ]:
print(f'before: {dt.shape[0]} observations, {dt.shape[1]} variables')
dt.drop(['ratio'],axis=1, inplace=True)

I = dt.income>1.5
dt.drop(dt[I].index, axis=0, inplace=True)

display(dt)

dt.reset_index(drop=True, inplace=True)

display(dt)

dt.drop(range(5), axis=0, inplace=True)
print(f'after: {dt.shape[0]} observations, {dt.shape[1]} variables')

**Answer:** see A08.py

## Renaming

Rename *consumption* to *cons* and *income* to *inc*.

In [ ]:
# write your code
dt.rename(columns={'consumption':'cons', 'income':'inc'}, inplace=True)
dt.head()

**Answer:** see A09.py

## Functions

Correct the wrong lines such that `assets_1 = assets_2 = assets_3 = assets_4`.

In [ ]:
def assets_row_by_row(x,R,Y):
    return (x.inc - x.cons)*R + Y
    
def assets_all_at_once(income,consumption,R,Y):
    return (income-consumption)*R + Y

def assets_adj(assets,R,Y):
    assets *= R
    assets += Y
    
    

R = 1.2 # return rate
Y = 1 # income
try:
    dt['assets_1'] = R*(dt['inc']-dt['cons'])+Y
    dt['assets_2'] = dt.apply(assets_row_by_row,axis=1,args=(R,Y))
    dt['assets_3'] = assets_all_at_once(dt['inc'].values,dt['cons'].values,R,Y)
    dt['assets_4'] = dt['inc']-dt['cons']
    assets_adj(dt['assets_4'],R,Y)
except:
    print('failed')
dt.head()    

**Answer:** see A10.py

# Problem

Load the data set in *data/NAH1_pivoted.xlsx* and clean and structure it such that the `plot_timeseries(dataframe)` below can be run and produce an interactive figure. 

In [ ]:
def _plot_timeseries(dataframe, variable, years):
    
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    dataframe.loc[:,['year']] = pd.to_numeric(dataframe['year'])
    I = (dataframe['year'] >= years[0]) & (dataframe['year'] <= years[1])
    
    x = dataframe.loc[I,'year']
    y = dataframe.loc[I,variable]
    ax.plot(x,y)
    
    ax.set_xticks(list(range(years[0], years[1] + 1, 5)))    
    
def plot_timeseries(dataframe):
    
    widgets.interact(_plot_timeseries, 
    dataframe = widgets.fixed(dataframe),
    variable = widgets.Dropdown(
        description='variable', 
        options=['Y','C','G','I','X','M'], 
        value='Y'),
    years=widgets.IntRangeSlider(
        description="years",
        min=1966,
        max=2018,
        value=[1980, 2018],
        continuous_update=False,
    )                 
); 

**Hint 1:** You can base your renaming on this dictionary:

In [ ]:
rename_dict = {}
rename_dict['P.1 Output'] = 'Y'
rename_dict['P.3 Final consumption expenditure'] = 'C'
rename_dict['P.3 Government consumption expenditure'] = 'G'
rename_dict['P.5 Gross capital formation'] = 'I'
rename_dict['P.6 Export of goods and services'] = 'X'
rename_dict['P.7 Import of goods and services'] = 'M'

**Hint 2:** You code should have the following structure:

In [ ]:
# a. load data set
nah1 = pd.read_excel('data/NAH1_pivoted.xlsx',  skiprows=2)

In [ ]:
# b. rename variables
nah1.rename(columns=rename_dict, inplace=True)
nah1.rename(columns={'Unnamed: 0':'year', 'Unnamed: 1':'prices'}, inplace=True)

In [ ]:
# c. remove rows where Y is nan
nah1.dropna(subset=['Y'], inplace=True)

In [ ]:
# d. correct year column data
I = nah1['prices'] == 'Current prices'
J = nah1['prices'] == '2010-prices, chained values'
nah1.loc[J,['year']] = nah1.loc[I,['year']].values

In [ ]:
# e. only keep rows with '2010-prices, chained values'
nah1_nom = nah1[I] # for use in the extra problem
nah1= nah1[J]

In [ ]:
# f. only keep renamed variables
nah1 = nah1[['year', 'prices'] + list(rename_dict.values())]
nah1_nom = nah1_nom[['year', 'prices'] + list(rename_dict.values())] 

**Answer:** see A11.py

In [ ]:
# g. interactive plot
plot_timeseries(nah1)

# Extra problems

## Extend interactive plot

Extend the interactive plot with a choice of *real* vs *nominal*.

In [ ]:
# Note that earlier, I saved the variables in nominal prices in a df called nah1_nom
full_nah1 = pd.concat((nah1, nah1_nom), axis=0)

# define plotting function
def _plot_timeseries_new(dataframe, variable, years, prices): #include prices as input in helper function
    
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    dataframe.loc[:,['year']] = pd.to_numeric(dataframe['year'])
    I = (dataframe['year'] >= years[0]) & (dataframe['year'] <= years[1])
    
    I &= dataframe['prices'] == prices # select prices
    
    x = dataframe.loc[I,'year']
    y = dataframe.loc[I,variable]
    ax.plot(x,y)
    
    ax.set_xticks(list(range(years[0], years[1] + 1, 5)))


def plot_timeseries_new(dataframe):

    widgets.interact(_plot_timeseries_new, 
    dataframe = widgets.fixed(value=dataframe),
    prices = widgets.Dropdown(
        description='prices',
        options=['2010-prices, chained values', 'Current prices'],
        value='2010-prices, chained values'), #include prices in dropdown menu
    variable = widgets.Dropdown(
        description='variable', 
        options=['Y','C','G','I','X','M'], 
        value='Y'),
    years=widgets.IntRangeSlider(
        description="years",
        min=1966,
        max=2018,
        value=[1980, 2018],
        continuous_update=False,
    )                 
) 

plot_timeseries_new(full_nah1)

## New data set

Load data from an Excel or CSV file you have downloaded from e.g. [Statistikbanken.dk](https://www.statistikbanken.dk/). Clean, structure and present the data as you see fit.